In [25]:
import random
import os
ascii_art = {
    "rock": """
    ___
---'   __)
      (___)
      (__)
      (_)
---._(__)

    """,
    "paper": """
     ___
---'    _)_
            __)
           ___)
          ___)
---.____)

    """,
    "scissors": """
    ___
---'   _)_
          __)
       ____)
      (__)
---._(__)

    """
}

LEADERBOARD_FILE = "Leaderboard.txt" 

def load_leaderboard():
    leaderboard = {}
    if os.path.exists(LEADERBOARD_FILE):
         with open(LEADERBOARD_FILE, "r") as file:
             for line in file:
                line = line.strip()
                if line:  # Check if the line is not empty
                    parts = line.split(",")
                    if len(parts) == 3: 
                        name, wins, losses = parts
                        name=name.lower()
                        if wins.isdigit() and losses.isdigit():
                                leaderboard[name] = {"wins": int(wins), "losses": int(losses)}
                        else:
                                print("Skipping invalid entry:", line)
    return leaderboard


def save_leaderboard(leaderboard):
    with open(LEADERBOARD_FILE, "w") as file:
        for name, stats in leaderboard.items():
            file.write(name + "," + str(stats['wins']) + "," + str(stats['losses']) + "\n")



# Player class
class Player:
    def __init__(self, name):
        self.name = name
        self.wins = 0
        self.losses = 0

    def win_percentage(self): 
        total_games = self.wins + self.losses
        if total_games > 0: 
            return (self.wins / total_games) * 100
        return 0  

# Game class
class RockPaperScissor:
    def welcome_user(self):  
        print("___________________________________________________________________________________________________________________")
        print("___________________________________________________________________________________________________________________")
        print("___________________________________________________________________________________________________________________")
        print("\033[31m--------------------------WELCOME TO THE GAME: ROCK, PAPER, SCISSORS--------------------------------\033[0m")
        print("___________________________________________________________________________________________________________________")
        print("___________________________________________________________________________________________________________________")
        print("___________________________________________________________________________________________________________________")
        print("\033[32m----------------------------------------GAME RULES-----------------------------------------------------\033[0m")
        print("\033[34m--> Rock wins against Scissors \033[0m")
        print("\033[34m--> Paper wins against Rock \033[0m")
        print("\033[34m--> Scissors wins against Paper \033[0m")
 
        
    def main_menu(self): 
        player_name = input("Enter your name: ").lower()
        player = Player(player_name)  
        self.welcome_user()
        while True:
            print("\nMain Menu:")
            print("1. Play Single Round")
            print("2. Play Best of N")
            print("3. View Leaderboard")
            print("4. Exit")
            choice = input("Enter your choice: ")
            
            if choice == "1":
                player_choice = self.get_player_choice()
                result = self.play_single_round(player_choice)
                if result == "win":
                    print("You won!")
                    player.wins += 1
                elif result == "lose":
                    print("You lost!")
                    player.losses += 1
                else:
                    print("It's a draw!")
                self.update_leaderboard(player)  
            elif choice == "2":
                n = int(input("Enter the number of rounds (e.g., 3, 5): "))
                self.play_best_of_n(player, n)
                self.update_leaderboard(player)  
            elif choice == "3":
                self.display_leaderboard()
            elif choice == "4":
                print("Thank you for playing!")
                break
            else:
                print("Invalid choice. Please try again.")

    def get_player_choice(self):  
        while True:
            print("-------------------------------------------------------------------------------------------------------")
            print("-------------------------------------------------------------------------------------------------------")
            print("                   \033[32m*************GAME STARTED******************\033[0m                          ")
            print("-------------------------------------------------------------------------------------------------------")
            print("-------------------------------------------------------------------------------------------------------")
            choice = input("Enter your choice (rock, paper, scissors): ").lower()
            if choice in ["rock", "paper", "scissors"]:
                return choice
            print("Invalid choice. Please try again.")

    def play_single_round(self, player_choice):  
        computer_choice = random.choice(["rock", "paper", "scissors"])  
        print("\nYour choice:\n" , ascii_art[player_choice])
        print("Computer's choice:\n" ,ascii_art[computer_choice])

        if player_choice == computer_choice:
            return "draw"
        if (player_choice == "rock" and computer_choice == "scissors") or \
           (player_choice == "paper" and computer_choice == "rock") or \
           (player_choice == "scissors" and computer_choice == "paper"):
            return "win"
        return "lose"

    def play_best_of_n(self, player, n):  
        wins = 0
        losses = 0
        
        for i in range(n):
            print("\nRound ", i + 1, "/",n)  
            player_choice = self.get_player_choice()  
            result = self.play_single_round(player_choice) 
            
            if result == "win":
                print("You won this round!")
                wins += 1
            elif result == "lose":
                print("You lost this round!")
                losses += 1
            else:
                print("This round is a draw!")

            if wins > n // 2 or losses > n // 2:
                break
        
        player.wins += wins
        player.losses += losses
        if wins > losses:
            print("\nYou won the game!")
        elif losses > wins:
            print("\nYou lost the game!")
        else:
            print("\nThe game is a draw!")

    def update_leaderboard(self, player):
        leaderboard = load_leaderboard()
        player_name = player.name.lower()
        if player.name in leaderboard:
            leaderboard[player.name]["wins"] += player.wins
            leaderboard[player.name]["losses"] += player.losses
        else:
            leaderboard[player.name] = {"wins": player.wins, "losses": player.losses}
        
        save_leaderboard(leaderboard)
        print("Leaderboard updated for" , player.name.capitalize() ,"!")
    
    def display_leaderboard(self):
        leaderboard = load_leaderboard()

        if not leaderboard:
            print("No leaderboard data found.")
            return

        print("\nLeaderboard:")
        print("Rank       Name          Wins     Losses       Win %")
        print("-" * 60)  
    
        def get_wins(item):
            return item[1]["wins"]

        sorted_leaderboard = sorted(leaderboard.items(), key=get_wins, reverse=True)
        rank = 1
        for name, stats in sorted_leaderboard:
            wins = stats["wins"]
            losses = stats["losses"]
            total_games = wins + losses
            win_percentage = (wins / total_games * 100) if total_games > 0 else 0  
            name_length = len(name.capitalize())
            name_spaces = 15 - name_length
            rank_str = str(rank) + "."
            rank_spaces = 4 - len(rank_str)
            print("  " + str(rank) + " "*rank_spaces + name.capitalize() + " " * name_spaces + str(wins) + "         " + str(losses) + "          " + str(win_percentage))
            rank += 1

# Run the game
if __name__ == "__main__":
    game = RockPaperScissor()
    game.main_menu()

Enter your name:  iqra


___________________________________________________________________________________________________________________
___________________________________________________________________________________________________________________
___________________________________________________________________________________________________________________
--------------------------WELCOME TO THE GAME: ROCK, PAPER, SCISSORS--------------------------------
___________________________________________________________________________________________________________________
___________________________________________________________________________________________________________________
___________________________________________________________________________________________________________________
----------------------------------------GAME RULES-----------------------------------------------------
--> Rock wins against Scissors 
--> Paper wins against Rock 
--> Scissors wins against Paper 

Main

Enter your choice:  3



Leaderboard:
Rank       Name          Wins     Losses       Win %
------------------------------------------------------------
  1  Momna          7         8          46.666666666666664
  2  Durre          3         1          75.0
  3  Amara          3         0          100.0
  4  Rumesah        2         1          66.66666666666666
  5  Ruhma          1         0          100.0
  6  Hira           1         2          33.33333333333333
  7  Amber          0         4          0.0
  8  Rabia          0         0          0
  9  Maryam         0         1          0.0
  10 Aymen          0         0          0
  11 Iqra           0         1          0.0
  12 Safoora        0         1          0.0

Main Menu:
1. Play Single Round
2. Play Best of N
3. View Leaderboard
4. Exit


Enter your choice:  4


Thank you for playing!
